# `Conflation`

In [89]:
#| default_exp conflate

In [664]:
#| export
import numpy as np, pandas as pd, scipy.sparse as sp, torch, json, os, re, math
import matplotlib.pyplot as plt
from IPython.display import display

from tqdm.auto import tqdm
from typing import Optional, Union, Dict, List
from IPython.display import display
from xclib.utils.sparse import retain_topk
from collections import Counter

## Helper functions

In [500]:
#| export
def display_items(matrix:sp.csr_matrix, info:Dict, n_data:Optional[int]=10, n_view_data:Optional[int]=20, 
                          seed:Optional[int]=1000):
    np.random.seed(seed)
    rnd_idx = np.random.permutation(matrix.shape[0])[:n_data]
    
    outputs = list()
    for idx in rnd_idx:
        sort_idx = np.argsort(matrix[idx].data)[:-n_view_data:-1]
        scores = matrix[idx].data[sort_idx]
        indices = matrix[idx].indices[sort_idx]
        labels = [info['text'][i] for i in indices]
        o = {
            "Substring": info['text'][idx],
            "Predictions": [(x, float(y)) for x,y in zip(labels, scores)]
        }
        outputs.append(o)
    return outputs
    

In [501]:
#| export
def compare_items(output_1, output_2):
    rows, index = [], []
    for x,y in zip(linker_output, pretrn_output):
        assert x["Substring"] == y["Substring"]
    
        index.extend([(x["Substring"], i+1) for i in range(len(x["Predictions"]))])
        rows.extend([(a[0], b[0]) for a,b in zip(x["Predictions"], y["Predictions"])])
        
    df = pd.DataFrame(
        rows,
        index=pd.MultiIndex.from_tuples(
            index,
            names=["Query", "Index"],
        ),
        columns=["Output 1", "Output 2"],
    )
    return df
    

In [497]:
#| export
def get_conflated_text(clusters:List, meta_info:Dict):
    texts = [[meta_info["text"][i] for i in c] for c in clusters]
    idx = np.argsort([len(c) for c in clusters])[::-1]
    return [texts[i] for i in idx]
        

In [683]:
#| export
class Operations:

    @staticmethod
    def invert(a):
        return np.divide(1.0, a, out=np.zeros_like(a), where=a!=0)

    @staticmethod
    def normalize(a):
        return a.multiply(np.sqrt(Operations.invert(a.sum(axis=1)))).multiply(np.sqrt(Operations.invert(a.sum(axis=0)))).tocsr()
        
    @staticmethod
    def compute_yty(mat:sp.csr_matrix, bsz:Optional[int]=1000, normalize=True):
        mat_t = mat.transpose()
        out = sp.vstack([mat[i:i+bsz]@mat_t for i in tqdm(range(0, mat.shape[0], bsz))])
        return Operations.normalize(out)

    @staticmethod
    def minimum(matrix:sp.csr_matrix):
        return np.array([matrix.data[i:j].min() for i,j in zip(matrix.indptr, matrix.indptr[1:])])

    @staticmethod
    def min_clamp(matrix:sp.csr_matrix, value:float, inplace:Optional[bool]=True):
        if not inplace:
            matrix = matrix.copy()
        matrix.data[matrix.data <= value] = 0
        matrix.eliminate_zeros()
        return matrix

    @staticmethod
    def max_clamp(matrix:sp.csr_matrix, value:float, inplace:Optional[bool]=True):
        if not inplace:
            matrix = matrix.copy()
        matrix.data[matrix.data >= value] = 0
        matrix.eliminate_zeros()
        return matrix

    @staticmethod
    def diff_threshold(matrix:sp.csr_matrix, value:float):
        diag = matrix.diagonal()
        matrix.setdiag(0)
        
        for i,j in zip(matrix.indptr, matrix.indptr[1:]):
            data = matrix.data[i:j]
            if len(data):
                idx = np.where(data.max() - data > value)[0]
                data[idx] = 0.0
                matrix.data[i:j] = data
        matrix.eliminate_zeros()
        
        matrix.setdiag(diag)
        return matrix

    @staticmethod
    def get_clusters(groups:np.array, lbl_idx:Optional[np.array]=None):
        lbl_idx = np.arange(groups.shape[0]) if lbl_idx is None else lbl_idx
        assert groups.shape == lbl_idx.shape, "`groups` and `lbl_idx` must have the same shape."
        clusters = dict()
        for k,v in zip(groups, lbl_idx): 
            clusters.setdefault(k, []).append(v)
        clusters = list(clusters.values())
        
        cluster_sz = [len(c) for c in clusters]
        sort_idx = np.argsort(cluster_sz)[::-1]
        return [clusters[i] for i in sort_idx]
    

In [516]:
#| export
class Filter:

    @staticmethod
    def from_max_size(clusters:List, size:int):
        return [c for c in clusters if len(c) < size]

    @staticmethod
    def from_min_size(clusters:List, size:int):
        return [c for c in clusters if len(c) > size]

    @staticmethod
    def remove_top_clusters(clusters:List, topk:int):
        sort_idx = np.argsort([len(c) for c in clusters])[-topk-1::-1]
        return [clusters[i] for i in sort_idx]
        

## Load data

In [95]:
data_dir = "/Users/suchith720/Projects/data/beir/msmarco/XC/substring/"

trn_file = f"{data_dir}/substring_trn_X_Y.npz"
tst_file = f"{data_dir}/substring_tst_X_Y.npz"
info_file = f"{data_dir}/raw_data/substring.raw.csv"

trn_meta, tst_meta = sp.load_npz(trn_file), sp.load_npz(tst_file)
meta_info = pd.read_csv(info_file)

meta_phrases = sp.load_npz(f"{data_dir}/derived-phrases_substring_X_Y.npz")

In [109]:
linker_dir = "/Users/suchith720/Downloads/00_msmarco-gpt-concept-substring-linker-with-ngame-loss-001/"

In [100]:
with pd.option_context("display.max_colwidth", None):
    display(meta_info.head())

,identifier,text
0,0,success of the Manhattan Project
1,1,hundreds of thousands of innocent lives obliterated
2,2,Restorative justice
3,3,amber
4,4,shades of yellow


## Clustering

### `Semantic:` Connected components

In [431]:
lnk_meta_mat = retain_topk(sp.load_npz(f"{linker_dir}/test_predictions_labels.npz"), k=10)
ptr_meta_mat = retain_topk(sp.load_npz(f"{linker_dir}/test_predictions_labels_msmarco-distilbert-cos-v5.npz"), k=10)

In [432]:
lnk_meta_mat.shape, ptr_meta_mat.shape

((1657190, 1657190), (1657190, 1657190))

In [122]:
linker_output = display_similar_items(lnk_meta_mat, meta_info, n_data=10, n_view_data=20, seed=1000)
pretrn_output = display_similar_items(ptr_meta_mat, meta_info, n_data=10, n_view_data=20, seed=1000)

In [170]:
df = compare_items(linker_output, pretrn_output)

In [433]:
with pd.option_context('display.max_rows', None):
    display(df)

Output 1  \
Query                                          Index                                                      
fascial aches, pains, tension and restrictions 1                                       investing fascia   
                                               2                                        spasm-type pain   
                                               3         fascial aches, pains, tension and restrictions   
                                               4                                 restrictions in fascia   
                                               5                      osteoarthritis-related joint pain   
                                               6                   Relieving pain of normal intact skin   
                                               7                            muscles, ligaments, tendons   
                                               8                                      contours the body   
                                               9                              deeper and thicker fascia   
                                               10        penetrates deep into aching muscles and joints   
various branches of engineering                1       highly specialized professional technical degree   
                                               2      study of the theory, experimentation, and engi...   
                                               3      conceive, design, build, supervise, operate, c...   
                                               4      engineer who is registered with the Engineerin...   
                                               5                                  branch of engineering   
                                               6                                        Civil engineers   
                                               7      oldest, broadest and largest of all other engi...   
                                               8      application of science and engineering principles   
                                               9      develop ways to utilize economically the mater...   
                                               10           Principles and Practice of Engineering (PE)   
average number of moves is 40.04               1                           target number to roll is a 1   
                                               2                                               50 moves   
                                               3                                     magic number at 60   
                                               4                           500 meters on a point target   
                                               5                                  price action analysis   
                                               6                  how many units they are likely to buy   
                                               7                       average litter size is 3 puppies   
                                               8                                       20 moves or less   
                                               9                                               1 Putter   
                                               10                                more liquid the market   
another four minutes                           1                             cook for another 2 minutes   
                                               2                                       sear the outside   
                                               3                                  take up to 30 minutes   
                                               4            Let stand, still covered, another 3 minutes   
                                               5                        grill until you get a good sear   
                                               6                                 2 minutes on each side   
                             

In [637]:
def from_similarity(lbl_lbl:sp.csr_matrix, score_thresh:Optional[float]=0.3, diff_thresh:Optional[float]=0.2, 
                    sim_topk:Optional[int]=20, directed:Optional[bool]=True, connection:Optional[str]="weak"):

    lbl_lbl = retain_topk(lbl_lbl, k=20)
    lbl_lbl = Operations.diff_threshold(lbl_lbl, diff_thresh)
    lbl_lbl = Operations.normalize(lbl_lbl)

    lbl_lbl.setdiag(0)
    lbl_lbl = retain_topk(lbl_lbl, k=sim_topk)

    n, groups = sp.csgraph.connected_components(lbl_lbl, directed=directed, connection=connection)
    return Operations.get_clusters(groups)
    

In [622]:
n, groups = sp.csgraph.connected_components(meta_mat, directed=True, connection='weak')
clusters = Operations.get_clusters(groups)

### `Predictions`

In [124]:
model_dir = "/Users/suchith720/Downloads/00_msmarco-gpt-concept-substring-linker-with-ngame-loss-001/"
data_meta = sp.load_npz(f"{model_dir}/train_predictions.npz")

In [73]:
def from_predictions(data_lbl:sp.csr_matrix, score_thresh:Optional[float]=0.3, diff_thresh:Optional[float]=0.2, 
                     pred_topk:Optional[int]=3):
    data_lbl = retain_topk(data_lbl, k=pred_topk)
    data_lbl = Operations.min_clamp(data_lbl, score_thresh)
    data_lbl = Operations.diff_threshold(data_lbl, diff_thresh)
    
    lbl_mat = Operations.compute_yty(data_lbl.transpose())
    n, groups = sp.csgraph.connected_components(lbl_mat)
    return Operations.get_clusters(groups)
    

### `Derived phrases`

In [207]:
def from_derived_phrases(lbl_phrases:sp.csr_matrix):
    lbl_mat = Operations.compute_yty(lbl_phrases)
    n, groups = sp.csgraph.connected_components(lbl_mat)
    return Operations.get_clusters(groups)
    

In [231]:
clusters = from_derived_phrases(meta_phrases)

  0%|          | 0/1658 [00:00<?, ?it/s]

In [209]:
texts = get_conflated_text(clusters, meta_info)

In [214]:
texts[2]

['high cholesterol',
 'cholesterol',
 'High Levels Of Fat In The Blood',
 'reduce LDL cholesterol in blood',
 'High blood cholesterol levels',
 'High cholesterol',
 'help you elevate your HDL',
 'raises HDL',
 'lower your cholesterol',
 'your LDL, or bad cholesterol',
 'improve your cholesterol numbers',
 'help lower cholesterol',
 'lower high cholesterol',
 'dyslipidemia',
 'lowering cholesterol levels',
 'increases the good HDL cholesterol',
 'to lower cholesterol',
 'LDL (bad) cholesterol',
 'low-density lipoprotein cholesterol (LDLc)',
 'lower blood cholesterol',
 'treat high cholesterol levels',
 'raise the level of cholesterol in the blood',
 'lower cholesterol',
 'reduce your cholesterol',
 'increased cholesterol levels',
 'treat high cholesterol',
 'increase bad cholesterol',
 'LDL cholesterol',
 'higher levels of bad cholesterol',
 'help maintain healthy cholesterol levels',
 'lower the bad cholesterol levels',
 'total cholesterol levels',
 'increased serum cholesterol',
 'kee

### Class cluster

In [653]:
#| export
class Cluster:

    @staticmethod
    def from_similarity(lbl_lbl:sp.csr_matrix, score_thresh:Optional[float]=0.3, diff_thresh:Optional[float]=0.2, 
                    sim_topk:Optional[int]=1, directed:Optional[bool]=True, connection:Optional[str]="weak"):
        lbl_lbl = lbl_lbl if diff_thresh is None else Operations.diff_threshold(lbl_lbl, diff_thresh)
        lbl_lbl = Operations.normalize(lbl_lbl)
    
        lbl_lbl.setdiag(0)
        lbl_lbl = lbl_lbl if sim_topk is None else retain_topk(lbl_lbl, k=sim_topk)
    
        n, groups = sp.csgraph.connected_components(lbl_lbl, directed=directed, connection=connection)
        return Operations.get_clusters(groups)

    @staticmethod
    def from_predictions(data_lbl:sp.csr_matrix, score_thresh:Optional[float]=0.3, diff_thresh:Optional[float]=0.2, 
                         pred_topk:Optional[int]=3, directed:Optional[bool]=True, connection:Optional[str]="weak"):
        data_lbl = retain_topk(data_lbl, k=pred_topk)
        data_lbl = Operations.min_clamp(data_lbl, score_thresh)
        data_lbl = Operations.diff_threshold(data_lbl, diff_thresh)
        
        lbl_mat = Operations.compute_yty(data_lbl.transpose())
        n, groups = sp.csgraph.connected_components(lbl_mat, directed=directed, connection=connection)
        return Operations.get_clusters(groups)

    @staticmethod
    def from_derived_phrases(lbl_phrases:sp.csr_matrix, directed:Optional[bool]=True, connection:Optional[str]="weak"):
        lbl_mat = Operations.compute_yty(lbl_phrases)
        n, groups = sp.csgraph.connected_components(lbl_mat, directed=directed, connection=connection)
        return Operations.get_clusters(groups)
        

## `Conflation`

In [718]:
#| export
class Conflation:

    @staticmethod
    def get_groups(clusters:List, n_lbl:int):
        groups, factor = np.full(n_lbl, -1), np.full(n_lbl, 1.0)
        for i,idx in enumerate(clusters):
            groups[idx] = i
            factor[idx] = 1/len(idx)
        idxs = np.where(groups == -1)[0]
        groups[idxs] = np.arange(len(clusters), len(clusters) + idxs.shape[0])
        return groups, factor

    @staticmethod
    def get_conflated_matrix(matrix:sp.csr_matrix, groups:np.array, n_lbl:Optional[int]=None, 
                             factor:Optional[np.array]=None):
        indptr = matrix.indptr

        if factor is not None:
            assert factor.shape == groups.shape
        
        data = matrix.data if factor is None else [d*factor[i] for d,i in zip(matrix.data, matrix.indices)]
        indices = [groups[i] for i in matrix.indices]
        shape = None if n_lbl is None else (matrix.shape[0], n_lbl)
        
        conflated_matrix = sp.csr_matrix((data, indices, indptr), shape=shape)
        conflated_matrix.sort_indices()
        conflated_matrix.sum_duplicates()
        return conflated_matrix

    @staticmethod
    def get_conflated_info(clusters:List, info:Dict):
        conflated_info, flag, lbl_idx = dict(), np.zeros(len(info["identifier"])), []
        
        for idxs in clusters:
            conflated_info.setdefault("identifier", []).append(" || ".join([str(info["identifier"][i]) for i in idxs]))
            conflated_info.setdefault("text", []).append(" [SEP] ".join([info["text"][i] for i in idxs if isinstance(info["text"][i], str)]))
            lbl_idx.append(idxs)
            flag[idxs] = 1
            
        idxs = np.where(flag == 0)[0]
        for idx in idxs:
            conflated_info.setdefault("identifier", []).append(info["identifier"][idx])
            conflated_info.setdefault("text", []).append(info["text"][idx])
            lbl_idx.append([idx])
            flag[idx] = 1
            
        assert flag.all(), "All items should be covered."
        return conflated_info, lbl_idx

    @staticmethod
    def perform_conflation(clusters:List, trn_meta:sp.csr_matrix, meta_info:Dict, tst_meta:Optional[sp.csr_matrix]=None, 
                           lbl_meta:Optional[sp.csr_matrix]=None):
        
        groups, _ = Conflation.get_groups(clusters, n_lbl=trn_meta.shape[1])

        conflated_trn_meta = Conflation.get_conflated_matrix(trn_meta, groups)
        conflated_meta_info, meta_idx = Conflation.get_conflated_info(clusters, meta_info)
        
        conflated_tst_meta = None if tst_meta is None else Conflation.get_conflated_matrix(tst_meta, groups, n_lbl=conflated_trn_meta.shape[1])
        conflated_lbl_meta = None if lbl_meta is None else Conflation.get_conflated_matrix(lbl_meta, groups, n_lbl=conflated_trn_meta.shape[1])

        return conflated_trn_meta, conflated_meta_info, meta_idx, conflated_tst_meta, conflated_lbl_meta
    

### Derived phrase `+` semantic conflation

In [684]:
clusters = Cluster.from_derived_phrases(meta_phrases)

  0%|          | 0/1658 [00:00<?, ?it/s]

In [687]:
clusters = Filter.remove_top_clusters(clusters, topk=1)

In [708]:
ctrn_meta, cmeta_info, meta_idx, ctst_meta, _ = Conflation.perform_conflation(clusters, trn_meta, meta_info, 
                                                                              tst_meta=tst_meta)

In [692]:
meta_file = f"{linker_dir}/test_predictions_labels_msmarco-distilbert-cos-v5.npz"
meta_mat = sp.load_npz(meta_file)

In [713]:
groups, factor = Conflation.get_groups(meta_idx, n_lbl=trn_meta.shape[1])
conflated_meta_mat = Conflation.get_conflated_matrix(meta_mat, groups, factor=factor)
conflated_meta_mat = conflated_meta_mat.T.tocsr()
conflated_meta_mat = Conflation.get_conflated_matrix(conflated_meta_mat, groups, factor=factor)

In [738]:
examples = []
for idx in np.random.permutation(conflated_meta_mat.shape[1])[:10]:
    sorted_idx = np.argsort(conflated_meta_mat[idx].data)[::-1][:5]
    indices = conflated_meta_mat[idx].indices[sorted_idx]
    
    examples.append({
        "Substring": cmeta_info["text"][idx],
        "Related": [cmeta_info["text"][i] for i in indices]
    })
    

In [739]:
print(json.dumps(examples, indent=4))

[
    {
        "Substring": "245 kilometers of Black Sea coastline",
        "Related": [
            "245 kilometers of Black Sea coastline",
            "26 kilometres (16 mi) from the modern coastline of the Aegean Sea",
            "Black Sea [SEP] Black Sea area [SEP] the Black Sea [SEP] Black Sea coastline [SEP] Black Sea Region [SEP] coast of Black Sea [SEP] Black Seas [SEP] The Black Sea"
        ]
    },
    {
        "Substring": "nearly 80 years ago",
        "Related": [
            "nearly 80 years ago",
            "about 100 years ago",
            "over 300 years ago",
            "more than 34 years ago",
            "about 170 years ago"
        ]
    },
    {
        "Substring": "$900 per month",
        "Related": [
            "$900 per month",
            "$850 per month",
            "introductory annual rate of $5 per month",
            "standard base rate of $735 per month",
            "$8000 a month mortgage payment"
        ]
    },
    {
        "Substri

In [742]:
clusters = Cluster.from_similarity(conflated_meta_mat, diff_thresh=0.2, sim_topk=1)

In [750]:
ctrn_meta, cmeta_info, meta_idx, ctst_meta, _ = Conflation.perform_conflation(clusters, ctrn_meta, cmeta_info, 
                                                                              tst_meta=ctst_meta)

In [744]:
conflated_text = get_conflated_text(clusters, meta_info=cmeta_info)

In [751]:
ctrn_meta.getnnz(axis=0).mean()

np.float64(5.610268714877188)

### Semantic conflation

In [650]:
meta_file = f"{linker_dir}/test_predictions_labels_msmarco-distilbert-cos-v5.npz"
meta_mat = retain_topk(sp.load_npz(meta_file), k=20)

In [651]:
clusters = Cluster.from_similarity(meta_mat, diff_thresh=0.2, sim_topk=1)

In [669]:
ctrn_meta, cmeta_info, meta_idx, ctst_meta, _ = Conflation.perform_conflation(clusters, trn_meta, meta_info, 
                                                                              tst_meta=tst_meta)

## `Save data`

In [753]:
#| export
class SaveData:

    @staticmethod
    def save_raw(fname:str, ids:List, txt:List):
        df = pd.DataFrame({"identifier": ids, "text": txt})
        df.to_csv(fname, index=False)

    @staticmethod
    def proc(save_dir:str, trn_file:str, trn_meta:sp.csr_matrix, info_file:str, meta_info:Dict, 
             tst_file:Optional[str]=None, tst_meta:Optional[sp.csr_matrix]=None, lbl_file:Optional[str]=None, 
             lbl_meta:Optional[sp.csr_matrix]=None, reuse_prev_dir:Optional[bool]=False):
        
        n = len([o for o in os.listdir(save_dir) if re.match(r'conflation_[0-9]{2}', o)])
        n = n if reuse_prev_dir else n + 1
        save_dir = f"{save_dir}/conflation_{n:02d}"
        
        raw_dir = f"{save_dir}/raw_data"
        os.makedirs(raw_dir, exist_ok=True)

        sp.save_npz(f"{save_dir}/{os.path.basename(trn_file)}", trn_meta)
        
        if tst_meta is not None:
            sp.save_npz(f"{save_dir}/{os.path.basename(tst_file)}", tst_meta)
            
        if lbl_meta is not None:
            sp.save_npz(f"{save_dir}/{os.path.basename(lbl_file)}", lbl_meta)

        raw_file = f"{raw_dir}/{os.path.basename(info_file)}"
        SaveData.save_raw(raw_file, meta_info["identifier"], meta_info["text"])
        

In [763]:
save_dir = "/Users/suchith720/Projects/data/beir/msmarco/XC/substring/"

In [764]:
SaveData.proc(save_dir, trn_file=trn_file, trn_meta=ctrn_meta, info_file=info_file, meta_info=cmeta_info, 
              tst_file=tst_file, tst_meta=ctst_meta)

## Driver

In [784]:
#| export
def load_data(data_dir:str, meta_type:str, linker_dir:str):
    trn_file = f"{data_dir}/{meta_type}_trn_X_Y.npz"
    tst_file = f"{data_dir}/{meta_type}_tst_X_Y.npz"
    lbl_file = f"{data_dir}/{meta_type}_lbl_X_Y.npz"
    
    info_file = f"{data_dir}/raw_data/{meta_type}.raw.csv"
    
    
    trn_meta = None if trn_file is None else sp.load_npz(trn_file)
    tst_meta = None if tst_file is None else sp.load_npz(tst_file)
    lbl_meta = None if lbl_file is None else sp.load_npz(lbl_file)
    
    meta_info = pd.read_csv(info_file)
    
    meta_phrases = sp.load_npz(f"{data_dir}/derived-phrases_{meta_type}_X_Y.npz")

    return (trn_meta, tst_meta, lbl_meta), meta_info, meta_phrases
    

In [789]:
#| export
def _01_perform_similarity_based_conflation_01(meta_file:str, trn_meta:sp.csr_matrix, tst_meta:sp.csr_matrix, 
                                        lbl_meta:sp.csr_matrix, meta_info:Dict):
    meta_mat = retain_topk(sp.load_npz(meta_file), k=20)
    clusters = Cluster.from_similarity(meta_mat, diff_thresh=0.2, sim_topk=1)
    
    return Conflation.perform_conflation(clusters, trn_meta, meta_info, tst_meta=tst_meta, lbl_meta=lbl_meta)
    

In [790]:
#| export
def _02_perform_phrase_based_conflation(meta_phrases:sp.csr_matrix, meta_file:str, trn_meta:sp.csr_matrix, 
                                    tst_meta:sp.csr_matrix, lbl_meta:sp.csr_matrix, meta_info:Dict):
    clusters = Cluster.from_derived_phrases(meta_phrases)
    clusters = Filter.remove_top_clusters(clusters, topk=1)

    output = Conflation.perform_conflation(clusters, trn_meta, meta_info, tst_meta=tst_meta, lbl_meta=lbl_meta)
    ctrn_meta, cmeta_info, meta_idx, ctst_meta, clbl_meta = output

    meta_mat = sp.load_npz(meta_file)
    groups, factor = Conflation.get_groups(meta_idx, n_lbl=trn_meta.shape[1])
    assert groups.max() == ctrn_meta.shape[1]
    
    conflated_meta_mat = Conflation.get_conflated_matrix(meta_mat, groups, factor=factor)
    conflated_meta_mat = conflated_meta_mat.T.tocsr()
    conflated_meta_mat = Conflation.get_conflated_matrix(conflated_meta_mat, groups, factor=factor)

    clusters = Cluster.from_similarity(conflated_meta_mat, diff_thresh=0.2, sim_topk=1)
    return Conflation.perform_conflation(clusters, ctrn_meta, cmeta_info, tst_meta=ctst_meta, lbl_meta=clbl_meta)
    

In [778]:
data_dir = "/Users/suchith720/Projects/data/beir/msmarco/XC/substring/"
linker_dir = "/Users/suchith720/Downloads/00_msmarco-gpt-concept-substring-linker-with-ngame-loss-001/"
meta_type = "substring"

info_file = f"{data_dir}/raw_data/substring.raw.csv"
linker_dir = "/Users/suchith720/Downloads/00_msmarco-gpt-concept-substring-linker-with-ngame-loss-001/"
meta_file = f"{linker_dir}/test_predictions_labels_msmarco-distilbert-cos-v5.npz"

In [780]:
save_dir = "/Users/suchith720/Projects/data/beir/msmarco/XC/substring/"

In [772]:
(trn_meta, tst_meta, lbl_meta), meta_info, meta_phrases = load_data(data_dir, meta_type, linker_dir)

output = _01_perform_similarity_based_conflation_01(meta_file, trn_meta, tst_meta, lbl_meta, meta_info)

output = _02_perform_phrase_based_conflation(meta_phrases, meta_file, trn_meta, tst_meta, lbl_meta, meta_info)

ctrn_meta, cmeta_info, meta_idx, ctst_meta, clbl_meta = output

SaveData.proc(save_dir, trn_file=trn_file, trn_meta=ctrn_meta, info_file=info_file, meta_info=cmeta_info, 
                  tst_file=tst_file, tst_meta=ctst_meta)